In [1]:
#%%
%load_ext autoreload
%autoreload 2

In [4]:
import solution

df = await solution.get_dataframe()
display(df)

,ticker,Previous Close
0,MMM,89.05
1,AOS,68.63
2,ABT,96.76
3,ABBV,149.11
4,ACN,312.01
5,ATVI,94.34
6,ADM,73.90
7,ADBE,529.29
8,ADP,250.18
9,AES,12.68
